In [13]:
import os, errno, sys
from bs4 import BeautifulSoup
from requests import get
from textblob import TextBlob
import pandas as pd
from pyteaser import Summarize

proj_path = "/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/site/src/"
# This is so Django knows where to find stuff.
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "manatwee.settings")
sys.path.append(proj_path)

# This is so my local_settings.py gets loaded.
os.chdir(proj_path)

# This is so models get loaded.
from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()

from twttr.models import *
from critic.models import *

In [14]:
NYTOrg = Organization.objects.get(name='NYT')
def scrapeNYTMovie():
    review_dict = {}
    url = 'https://www.nytimes.com/reviews/movies'
    html_soup=BeautifulSoup(get(url).text,'html.parser')
    movie_length = len(html_soup.find_all('a',class_='story-link'))
    print movie_length
    for i in range(movie_length):
        souplet = html_soup.find_all('a',class_='story-link')[i]
        r_url = 'https://www.nytimes.com/'+souplet['href']
        r_name = souplet.h2.text.strip()
        r_get = get(r_url)
        r_soup = BeautifulSoup(r_get.text,'html.parser')
        review_text = ""
        chunk_count = len(r_soup.find_all('p',class_='story-body-text story-content'))
        author = r_soup.find_all('meta',attrs={"name":"author"})[0]['content']
        for i in range(chunk_count):
            review_text += r_soup.find_all('p',class_='story-body-text story-content')[i].text
        review_sentiment = TextBlob(review_text).sentiment
        review_polarity=review_sentiment[0]
        review_subjectivity=review_sentiment[1]
        r_thumbsup= 'Positive' if review_polarity > 0 else 'Negative'
        summary = Summarize(r_name,review_text)
        r_blurb = ' '.join(summary)
        review_dict[souplet.h2.text.strip()] = {'thumbsup':r_thumbsup,'blurb':r_blurb,'name':r_name,'link':r_url,'text':review_text,'review_subjectivity':review_subjectivity,'critic':author,'organization':'NYT','review_polarity':review_polarity}
    return review_dict

In [15]:
review_dict = {}
url = 'https://www.nytimes.com/reviews/movies'
html_soup=BeautifulSoup(get(url).text,'html.parser')

In [16]:
movie_length = len(html_soup.find_all('a',class_='story-link'))


In [17]:
souplet = html_soup.find_all('a',class_='story-link')[0]

In [18]:
souplet

<a class="story-link" data-rref="" href="/2017/12/28/movies/animation-show-of-shows-quad-cinema.html">\n<div class="story-meta">\n<h2 class="headline" itemprop="headline">\n                    The 19th Annual Animation Show of Shows                </h2>\n<ul class="review-meta-menu">\n<li>\n<span itemprop="genre">Animation</span> </li>\n<li>Directed by\n                                                    <span itemprop="director" itemscope="" itemtype="http://schema.org/Person">Quentin Baillieux</span>,                                                     <span itemprop="director" itemscope="" itemtype="http://schema.org/Person">Lia Bertels</span>,                                                     <span itemprop="director" itemscope="" itemtype="http://schema.org/Person">Jac Clinch</span>,                                                     <span itemprop="director" itemscope="" itemtype="http://schema.org/Person">Pete Docter</span>,                                                    

In [10]:
r_url = 'https://www.nytimes.com/'+souplet['href']


In [12]:
scrapeNYTMovie()

20


{u'A German Life': {'blurb': u'\u201cA German Life\u201d is likely to be the last new movie of its kind: a documentary that presents contemporary testimony from someone who witnessed the inner workings of the Nazi high command. Pomsel, who is remarkably lucid for a centenarian, are intercut with quotations from Goebbels and archival footage from both sides of the war. Yet one of the recurring characters in her stories is a Jewish friend, Eva L\xf6wenthal, whom records indicate was deported to Auschwitz and, in 1945, declared dead.Ms. The movie is taken from 30 hours of interviews with Brunhilde Pomsel, who served as a secretary to Joseph Goebbels and was 103 when the film was shot. (She died in January at 106.)Like \u201cBlind Spot: Hitler\u2019s Secretary,\u201d which centered on the recollections of Traudl Junge, \u201cA German Life\u201d lets viewers consider its subject \u2014 her anecdotes, body language and face (often shot in close-up) \u2014 and decide how much to believe her. 